In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import sentencepiece as spm
import os

# 데이터 로드
trn = pd.read_csv('/aiffel/aiffel/destination_folder/nsmc-master/ratings_train.txt', sep='\t', quoting=3)
tst = pd.read_csv('/aiffel/aiffel/destination_folder/nsmc-master/ratings_test.txt', sep='\t', quoting=3)

# SentencePiece 모델 학습
corpus_file = '/aiffel/aiffel/destination_folder/nsmc-master/ratings_corpus.txt'
trn['document'].to_csv(corpus_file, index=False, header=False)
spm.SentencePieceTrainer.train('--input={} --model_prefix=spm_model --vocab_size=10000'.format(corpus_file))

# SentencePiece 모델 로드
sp = spm.SentencePieceProcessor()
sp.load('spm_model.model')

# 데이터 전처리
trn['document'] = trn['document'].astype(str)
tst['document'] = tst['document'].astype(str)

trn_text = trn['document'].apply(lambda x: sp.encode_as_ids(x)).tolist()
tst_text = tst['document'].apply(lambda x: sp.encode_as_ids(x)).tolist()

trn_text = pad_sequences(trn_text, padding='post')
tst_text = pad_sequences(tst_text, padding='post')



sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=/aiffel/aiffel/destination_folder/nsmc-master/ratings_corpus.txt --model_prefix=spm_model --vocab_size=10000
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /aiffel/aiffel/destination_folder/nsmc-master/ratings_corpus.txt
  input_format: 
  model_prefix: spm_model
  model_type: UNIGRAM
  vocab_size: 10000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0


In [8]:
# 모델 학습
embedding_dim = 128
hidden_units = 128

model = Sequential()
model.add(Embedding(len(sp), embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(trn_text, trn['label'], epochs=5, batch_size=64)

# 모델 평가
loss, accuracy = model.evaluate(tst_text, tst['label'], verbose=1)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)


Epoch 1/5
2344/2344 [==============================] - 21s 8ms/step - loss: 0.6932 - accuracy: 0.5012
Epoch 2/5
2344/2344 [==============================] - 19s 8ms/step - loss: 0.6932 - accuracy: 0.4995
Epoch 3/5
2344/2344 [==============================] - 19s 8ms/step - loss: 0.6932 - accuracy: 0.5020
Epoch 4/5
2344/2344 [==============================] - 20s 8ms/step - loss: 0.6932 - accuracy: 0.5005
Epoch 5/5
1563/1563 [==============================] - 6s 3ms/step - loss: 0.6932 - accuracy: 0.4965
Test Loss: 0.6931642889976501
Test Accuracy: 0.49654000997543335


In [18]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import re
from konlpy.tag import Mecab

# 데이터 로드
trn = pd.read_csv('/aiffel/aiffel/destination_folder/nsmc-master/ratings_train.txt', sep='\t', quoting=3)
tst = pd.read_csv('/aiffel/aiffel/destination_folder/nsmc-master/ratings_test.txt', sep='\t', quoting=3)

# 특수 문자 및 불필요한 문자 제거
def clean_text(text):
    text = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z\\s]", "", text)
    text = re.sub("\\s+", " ", text)
    text = text.strip()
    return text

# 불용어 제거
def remove_stopwords(text, stopwords):
    tokens = text.split()
    tokens = [token for token in tokens if token not in stopwords]
    text = ' '.join(tokens)
    return text

# 정규화
def normalize(text, tokenizer):
    tokens = tokenizer.morphs(text)
    text = ' '.join(tokens)
    return text

# 불용어 리스트 생성
stopwords = ['은', '는', '이', '가', '을', '를']

# 토크나이저 생성
tokenizer = Mecab()

# 데이터 전처리
trn['document'] = trn['document'].apply(lambda x: clean_text(str(x)))
trn['document'] = trn['document'].apply(lambda x: remove_stopwords(x, stopwords))
trn['document'] = trn['document'].apply(lambda x: normalize(x, tokenizer))

tst['document'] = tst['document'].apply(lambda x: clean_text(str(x)))
tst['document'] = tst['document'].apply(lambda x: remove_stopwords(x, stopwords))
tst['document'] = tst['document'].apply(lambda x: normalize(x, tokenizer))

# 데이터 확인
print("Train Data - First 5 rows:")
print(trn.head())

print("\nTest Data - First 5 rows:")
print(tst.head())

# 정수 인코딩
vocab_size = 10000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(trn['document'])

trn_text = tokenizer.texts_to_sequences(trn['document'])
tst_text = tokenizer.texts_to_sequences(tst['document'])

# 패딩
max_len = 30
trn_text = pad_sequences(trn_text, maxlen=max_len)
tst_text = pad_sequences(tst_text, maxlen=max_len)

# 모델 구성
model = Sequential()
model.add(Embedding(vocab_size, 128))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일 및 학습
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(trn_text, trn['label'], epochs=5, batch_size=64, validation_split=0.2)

# 모델 평가
loss, accuracy = model.evaluate(tst_text, tst['label'], verbose=2)
print("Test Accuracy:", accuracy)


Train Data - First 5 rows:
         id                                           document  label
0   9976970                               아 더 빙 진짜 짜증 나 네요 목소리      0
1   3819312                흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍 지 않 구나      1
2  10265843                                너무 재 밓었다그래서보는것을추천한다      0
3   9045019                      교도소 이야기 구먼 솔직히 재미 는 없 다 평점 조정      0
4   6483659  사이몬페그 의 익살 스런 연기 가 돋보였 던 영화 스파이더맨 에서 늙 어 보이 기 ...      1

Test Data - First 5 rows:
        id                                           document  label
0  6270596                                                굳 ㅋ      1
1  9274899                               GDNTOPCLASSINTHECLUB      0
2  8544678           뭐 야 평점 들 은 나쁘 진 않 지만 10 점 짜리 는 더더욱 아니 잖아      0
3  6825595                지루 하 지 는 않 은데 완전 막장 임 돈 주 고 보 기 에 는      0
4  6723715  3 D 만 아니 었 어도 별 다섯 개 줬 을 텐데 왜 3 D 로 나와서 제 심기 를...      0
Epoch 1/5
1875/1875 [==============================] - 11s 5ms/step - loss: 0.3776 - accuracy: 0.8301 - val_loss